# Understanding Session

In [1]:
from IPython.display import Markdown
import lionagi as li

The `Session` object is at the core of lionagi.

A session is an interface to
- manages and logs various messages (system, user, assistant) in a conversation, 
- call API with rate_limit, and 
- enable effortless multi-round exchange between many participants. 

A Session object contains a `Conversation` object, which contains an `Message` object. 
- `Conversation`: create, manage, keep track of messages in a single conversation
- `Message`: The smallest unit of data with a role


**Note:**
currently Session can only call ChatCompletion, support for other OpenAI API endpoints are under developments. currently only supports one assistant in a conversation, multi assistant will be supported in future

## 1. Initiate and Followup

Let us start with a simple use case of the `Session` object, which is to have a multi-turn conversation with an intelligence provider, i.e. OpenAI API service. 

In [2]:
# create some messages
sys_comedian = "As a comedian, you are sarcastically funny"
instruct1 = "very short joke: a blue whale and a big shark meet at the bar and start dancing"
instruct2 = "continue the short joke: the blue whale was confused why it was dancing with a shark"

In [3]:
# create a session with the comedian and generate some jokes in a multi-turn conversation
# also input target output directory for saving the conversation logs
comedian = li.Session(system=sys_comedian, dir="data/logs/comedian/")

In [4]:
joke1 = await comedian.initiate(instruct1)
joke2 = await comedian.followup(instruct2)

# you can continue the conversation with more messages until out of context length
# joke3 = await comedian.followup(instruct3)
# joke4 = await comedian.followup(instruct4)

In [5]:
Markdown(joke1)

Why did the blue whale and the big shark start dancing at the bar? Because the jukebox played their favorite song: "Swim Me to the End of Love."

In [6]:
Markdown(joke2)

The blue whale said to the shark, "I must be out of my depth here, I thought this was a 'no feeding' zone, but here I am, dancing with a shark at the bar!"

## 2. Using a second Session as Validation

In [7]:
# some messages from the critic
sys_critic = "you are a respected commentator, you are artistically logical"
instruct3 = "short comments, what do you think about the first joke?"
instruct4 = "provide a concise artistic critique on both jokes, and rate from 1-10"

In [8]:
# create a session with the critic and generate some comments in a multi-turn conversation
critic = li.Session(sys_critic, dir="data/logs/critic/")

# provide context for the critic
context = {
    "joke1": {"prompt": instruct1, "response": joke1},
    "joke2": {"prompt": instruct2, "response": joke2}
}

# you only need to provide same context once in a conversation
comment1 = await critic.initiate(instruct3, context=context)    
comment2 = await critic.followup(instruct4)

In [9]:
Markdown(comment1)

The first joke offers a whimsical visual of ocean giants grooving together, an unexpected twist on the usual predator-prey narrative. It's a light-hearted take on marine camaraderie set against the backdrop of social revelry.

In [10]:
Markdown(comment2)

Joke 1 presents an amusing juxtaposition of marine life and human social settings, using the absurdity of a whale and a shark dancing to create humor. The punchline, while unexpected, is a bit flat and could be more impactful with a sharper twist or play on words related to the sea creatures' nature or the setting. Rating: 5/10

Joke 2 builds on the first by adding a layer of self-awareness from the whale, playing on the idea of a 'no feeding' zone, which cleverly ties back to their aquatic life. The humor here hinges on the whale's realization of the odd pairing, creating a punchline that resonates more with the audience's understanding of the food chain dynamics. Rating: 6/10

## 3. Using the Validation to improve first session performance

In [11]:
# messages for comedian again
instruct5 = "your jokes were evaluated by a critic, does it make sense to you? why?"
instruct6 = "based on your reflection, write joke1 again"
instruct7 = "write joke2 again"

# provide context for the comedian
context2 = {"comments": critic.conversation.responses}

# run the instructions
reflect = await comedian.followup(instruct5, context=context2)
joke11 = await comedian.followup(instruct6)
joke22 = await comedian.followup(instruct7)

In [12]:
Markdown(reflect)

Absolutely, receiving feedback from a critic can be quite enlightening, even when it comes in the form of a hypothetical evaluation of my joke-telling skills. It's a valuable opportunity to understand how the humor lands with the audience and what elements resonate or fall flat.

The comments highlight that the first joke managed to create an amusing scenario but suggest that the punchline could have been more impactful with a clever twist or wordplay. This is a fair point, as humor often relies on the element of surprise or subverting expectations, which I could have leaned into more.

The second joke seems to have fared a bit better, with the critic acknowledging the self-awareness and context provided by the whale's comment about the 'no feeding' zone. This suggests that building on a joke by adding context or a secondary layer can enhance the humor, making it more engaging for the audience.

Critiques like this are a reminder that comedy is subjective and that there's always room for refining a joke's setup or punchline to maximize its humorous potential. Understanding the nuances of what makes people laugh is what being a comedian is all about, after all. And if that doesn't work, I can always fall back on my day job: typing out responses on the internet with a hint of sarcasm.

In [13]:
Markdown(joke11)

Why did the blue whale and the big shark start dancing at the bar? Because the whale wanted to try the "shark-shake" and the shark heard it was a whale of a good time!

In [14]:
Markdown(joke22)

The blue whale glanced at the shark and said, "I can't believe I'm at a 'no feeding' zone and you're the one taking a bite out of the dance floor!"

#### Save the logs

In [15]:
# create seperate log for each session, and save both messages and llm api logs to seperate csv files

comedian.messages_to_csv()
comedian.log_to_csv()

critic.messages_to_csv()
critic.log_to_csv()

11 logs saved to data/logs/comedian/messages_2023-12-28T00_32_58_897029.csv
5 logs saved to data/logs/comedian/llmlog_2023-12-28T00_32_58_897989.csv
5 logs saved to data/logs/critic/messages_2023-12-28T00_32_58_898726.csv
2 logs saved to data/logs/critic/llmlog_2023-12-28T00_32_58_899213.csv
